In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


**Pipeline for Long-Form Question Answering (LFQA)**

In [ ]:
!unzip dataTwo.zip

Archive:  dataTwo.zip
   creating: dataTwo/
  inflating: dataTwo/1.txt           
  inflating: dataTwo/10.txt          
  inflating: dataTwo/100.txt         
  inflating: dataTwo/101.txt         
  inflating: dataTwo/102.txt         
  inflating: dataTwo/103.txt         
  inflating: dataTwo/104.txt         
  inflating: dataTwo/105.txt         
  inflating: dataTwo/106.txt         
  inflating: dataTwo/107.txt         
  inflating: dataTwo/108.txt         
  inflating: dataTwo/109.txt         
  inflating: dataTwo/11.txt          
  inflating: dataTwo/110.txt         
  inflating: dataTwo/111.txt         
  inflating: dataTwo/112.txt         
  inflating: dataTwo/113.txt         
  inflating: dataTwo/114.txt         
  inflating: dataTwo/115.txt         
  inflating: dataTwo/116.txt         
  inflating: dataTwo/117.txt         
  inflating: dataTwo/118.txt         
  inflating: dataTwo/119.txt         
  inflating: dataTwo/12.txt          
  inflating: dataTwo/120.txt         
  infl

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-r3xb5fne/farm-haystack_9c4ce934e8f648aea420170c239de375
  Resolved https://github.com/deepset-ai/haystack.git to commit b353b22fffb6d4abd7f478966b3e085a369abab9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.9/299.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): fi

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-r3xb5fne/farm-haystack_9c4ce934e8f648aea420170c239de375
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


## Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.
Example log message:
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/218_Olenna_Tyrell.txt
Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

### Document Store

FAISS is a library for efficient similarity search on a cluster of dense vectors.
The `FAISSDocumentStore` uses a SQL(SQLite in-memory be default) database under-the-hood
to store the document text and other meta data. The vector embeddings of the text are
indexed on a FAISS Index that later is queried for searching answers.
The default flavour of FAISSDocumentStore is "Flat" but can also be set to "HNSW" for
faster search at the expense of some accuracy. Just set the faiss_index_factor_str argument in the constructor.

In [ ]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=768, faiss_index_factory_str="Flat")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


### Cleaning & indexing documents

we  convert and index documents to our DocumentStore

In [ ]:
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text


# Let's first get some files that we want to use
doc_dir = "/content/dataTwo"
# Convert files to dicts
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.preprocessing:Converting /content/dataTwo/17.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/466.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/303.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/433.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/183.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/238.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/23.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/148.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/47.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/268.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/248.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/272.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/3.txt
INFO:haystack.utils.preprocessing:Converting /content/dataTwo/126.txt
INFO:haystack.utils.prepr

Writing Documents:   0%|          | 0/564 [00:00<?, ?it/s]

### Initialize Retriever and Reader/Generator

#### Retriever

We use a `DensePassageRetriever` and we invoke `update_embeddings` to index the embeddings of documents in the `FAISSDocumentStore`



In [ ]:
save_dir="/content/drive/MyDrive/dpr"

In [ ]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever.load(load_dir=save_dir, document_store=document_store)

document_store.update_embeddings(retriever)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.nodes.retriever.dense:DPR model loaded from /content/drive/MyDrive/dpr
INFO:haystack.document_stores.faiss:Updating embeddings for 562 docs...


Updating Embedding:   0%|          | 0/562 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/576 [00:00<?, ? Docs/s]

Before we blindly use the `DensePassageRetriever` let's empirically test it to make sure a simple search indeed finds the relevant documents.

In [ ]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query="what is self join?", params={"Retriever": {"top_k": 3}})
print_documents(res, max_text_len=512)


Query: what is self join?

{   'content': 'self join is used to join a table with itself. it\n'
               'is used when the table has a foreign key that\n'
               'references its own primary key.\n'
               'cross join is used to combine each row from one\n'
               'table with every row from another table. it is\n'
               'also known as a cartesian join. the syntax of\n'
               'natural join is used to combine two tables based\n'
               'on columns that have the same name and data type.\n'
               'conclusion join operation is one of the most\n'
               'important operations in dbms. it is used to\n'
               'combine data from multiple tables and retrieve the\n'
               'requ...',
    'name': '187.txt'}

{   'content': 'a self join is used to join a table with itself.\n'
               'this join can be performed using table aliases,\n'
               'which allow us to avoid repeating the same table\n'
  

#### Reader/Generator

we now initalize our reader/generator.

Here we use a `Seq2SeqGenerator` with the bart_lfqa* model


In [ ]:
from haystack.nodes import Seq2SeqGenerator


generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


### Pipeline

With a Haystack `Pipeline` we can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs).

In [ ]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

**Ask Questions**

In [ ]:
def askQuestion(question):
  try:
    query = pipe.run(
        query=question, params={"Retriever": {"top_k": 3}}
    )
    answer = query['answers'][0].answer
    score = query['documents'][0].score

    result = {
        'question': question,
        'answer': answer,
        'score': score
    }

    return result
  except Exception:
    raise Exception("face not detected")

In [ ]:
question=  "What is a one-to-one relationship?"
askQuestion(question)

{'question': 'What is a one-to-one relationship?',
 'answer': 'A one-to-one relationship is where each record in one table is associated with exactly one record in another table, and vice versa.',
 'score': 0.6971657551119452}

In [ ]:
cd '/content/drive/MyDrive/pytorch-flask-api'

/content/drive/MyDrive/pytorch-flask-api


In [ ]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 30.0 MB/s eta 0:00:00


In [ ]:
from pymongo import MongoClient
# Replace <connection_string> with your modified MongoDB Atlas connection string
client = MongoClient("mongodb+srv://<user-name>:<password>@cluster0.fz59ulb.mongodb.net/studentDb?retryWrites=true&w=majority")

In [ ]:
# Replace <database_name> with the name of your database
db = client.get_database("studentDb")

In [ ]:
import requests
import os
import json

def download_image(url, filename, download_path):
    response = requests.get(url)
    if response.status_code == 200:
        filepath = os.path.join(download_path, filename)
        with open(filepath, 'wb') as file:
            file.write(response.content)
        return filepath
    return None

def download_images_from_mongodb():
    # Specify the download directory
    download_path = "/content/drive/MyDrive/pytorch-flask-api/persons"

    # Create the download directory if it doesn't exist
    os.makedirs(download_path, exist_ok=True)

    # Replace <collection_name> with the name of your collection
    collection = db.get_collection("students")

    # Find documents where uploadOnServer is False
    query = {"uploadOnServer": False}
    cursor = collection.find(query)

    # Iterate over the documents and download the images
    for document in cursor:
        name = document['name']
        student_id = str(document['_id'])
        image_url = document['image']

        # Create a JSON string representation of the object
        object_data = {
            "name": name,
            "student_id":student_id
        }
        json_str = json.dumps(object_data)

        # Use the JSON string as the filename
        filename = f"{json_str}.jpeg"

        # Download the image
        filepath = download_image(image_url, filename, download_path)

        if filepath:
            # Update the 'uploadOnServer' status to True
            query = {"_id": document['_id']}
            update = {"$set": {"uploadOnServer": True}}
            collection.update_one(query, update)
            print(f"Image downloaded and status updated for document: {document['_id']}")
            print(f"Image saved at: {filepath}")
        else:
            print(f"Image download failed for document: {document['_id']}")


In [ ]:
# Call the function to download images from MongoDB
download_images_from_mongodb()

In [ ]:
!pip install mongoengine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.6/110.6 kB 7.1 MB/s eta 0:00:00


In [ ]:
from datetime import datetime

In [ ]:
from bson.objectid import ObjectId
attendance_collection = db['attendances']
students_collection = db['students']
class Attendance:
    @staticmethod
    def create_attendance(student_id):
      try:
        # Check if the student_id exists in the students collection
        if not students_collection.count_documents({'_id': ObjectId(student_id)}):
            raise ValueError('Invalid student_id')
                # Get the current date and time
        current_time = datetime.now()
        attendance_doc = {
            'attendance_time': current_time,
            'student': student_id
        }
        attendance_collection.insert_one(attendance_doc)
      except Exception:
        raise Exception("face not detected")        

## **Install ngrok**

ngrok is cross-platform application that exposes local server ports to the Internet.

In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2023-05-23 05:10:43--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.35’

ngrok-stable-linux- 100%[===================>]  13.21M  11.8MB/s    in 1.1s    

2023-05-23 05:10:45 (11.8 MB/s) - ‘ngrok-stable-linux-amd64.tgz.35’ saved [13856790/13856790]



In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,270 kB]
Get:13 http://security.ubuntu.com/ub

Sign up link - https://dashboard.ngrok.com/signup?

In [ ]:
!ngrok authtoken <get-your-token>



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=79bbd2aec877ef939a8a3033157dcad5579ef9d567e5aab5c6782f6daf3ae726
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import face_recognition

In [ ]:
!pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 34.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=2e842b4dc09f80a275c31240e6ae956506ea8db7159b13f074b9e4f897c895af
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.2
    Uninstalling protobuf-3.20.2:
      Successfully uninstalled protobuf-3.20.2
ERROR: pip's d

In [ ]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [ ]:
import face_model
import argparse
import cv2
from time import sleep
import os
import unicodedata
import math
import numpy as np


def eu_dist(x, y):
    d = 0.0
    for i in range(len(x)):
        d += pow((float(x[i]) - float(y[i])), 2)
    d = math.sqrt(d)
    return d/len(x)


def save_encodings(imgs, name, model):
    global known_encodings, known_names
    for i in range(0, len(imgs)):
        im = cv2.imread(imgs[i])
        img, bbox = model.get_input(im, th=0.98)
        if img is not None:
            f2 = model.get_feature(img[0])
            known_encodings.append(f2)
            known_names.append(name[i])
        else:
            print('Warning: No face Detected for '+name[i]+'\n')


def str_to_ndarray(input):
    temp = input.split(',')
    temp = temp[0:len(temp)-1]
    for t in range(0, len(temp)):
        temp[t] = float(unicodedata.normalize(
            'NFKD', temp[t]).encode('ascii', 'ignore'))
    temp = np.asarray(temp)
    return temp


known_encodings = []
known_names = []


def get_encodings():
    global known_names
    global known_encodings
    known_encodings = []
    known_names = []
    qry = 'select name,encodings from encodings;'
    cnx = open_connection()
    if cnx:
        print('Getting encoding from Database')
        cursor = cnx.cursor()
        cursor.execute(qry)
        data = cursor.fetchall()
        for dat in data:
            name = dat[0]
            encodings = dat[1]
            known_encodings.append(str_to_ndarray(encodings))
            known_names.append(unicodedata.normalize(
                'NFKD', name).encode('ascii', 'ignore'))
        cursor.reset()
        close_connection(cnx)
        known_encodings = np.asarray(known_encodings)
        try:
            os.remove("encodings.npy")
            os.remove("names.npy")
            np.save('encodings.npy', known_encodings)
            np.save('names.npy', known_names)
            print('Latest Encodings are saved')
        except:
            print('Unable to store new encodings')
        return known_names, known_encodings
    else:
        print('Getting encodings from file')
        get_encodings_from_file()


def get_encodings_from_file():
    global known_names
    global known_encodings
    if os.path.exists('./encodings.npy') and os.path.exists('./names.npy'):
        known_encodings = np.load('encodings.npy')
        known_names = np.load('names.npy')
    return known_names, known_encodings


def face_recog(test_vec, th):
    dis = eu_dist(known_encodings[0], test_vec)
    min_dis = dis
    name = known_names[0]
    for vec in range(1, len(known_encodings)):
        dis = eu_dist(known_encodings[vec], test_vec)
        # print(known_names[vec])
        # print(dis)
        if dis < min_dis:
            min_dis = dis
            name = known_names[vec]
    if min_dis < th:
        return name
    else:
        return 'unknown'


def FaceMain(image):
    try:
      result = dict()
      global args, model

      img, bbox = model.get_input(image, th=0.98)
      if img is not None:
          for im in range(0, len(img)):
              f2 = model.get_feature(img[im])
              fhab = model.get_ga(img[im])
              name = face_recog(f2, 0.0021)
              gender, age = fhab
              result = {"name": name, "gender": gender, "age": age}
              return result
    except Exception:
      raise Exception("face not detected")


parser = argparse.ArgumentParser(description='face model test')
# # general
parser.add_argument('--image-size', default='112,112', help='')
parser.add_argument('--model', default='model-0000',
                    help='path to load model.')
parser.add_argument(
    '--ga-model', default='genderage_v1/model-0000', help='path to load model.')
parser.add_argument('--gpu', default=0, type=int, help='gpu id')
parser.add_argument('--det', default=0, type=int,
                    help='mtcnn option, 1 means using R+O, 0 means detect from begining')
parser.add_argument('--flip', default=0, type=int,
                    help='whether do lr flip aug')
parser.add_argument('--threshold', default=1.24,
                    type=float, help='ver dist threshold')
parser.add_argument("-f", "--file", required=False)
args = parser.parse_args()
model = face_model.FaceModel(args)
base_path = r'persons'
images_path = []
names_list = []
for im in os.listdir(base_path):
    images_path.append(os.path.join(base_path, im))
    names_list.append(im.split('.')[0])

if 'persons/.ipynb_checkpoints' in images_path:
    images_path.remove('persons/.ipynb_checkpoints')
if '' in names_list:
  names_list.remove('')
save_encodings(images_path, names_list, model)


['model', '0000']
loading model 0
['genderage_v1/model', '0000']
loading genderage_v1/model 0


In [ ]:
def download_and_append_encodings():
    download_images_from_mongodb()
    images_path = []
    names_list = []
    for im in os.listdir(base_path):
        images_path.append(os.path.join(base_path, im))
        names_list.append(im.split('.')[0])

    if 'persons/.ipynb_checkpoints' in images_path:
        images_path.remove('persons/.ipynb_checkpoints')
    if '' in names_list:
      names_list.remove('')
    save_encodings(images_path, names_list, model)



In [ ]:
from glob import glob
from keras import backend as K
import imutils
from keras.models import load_model
import numpy as np
import keras
import requests
from scipy.spatial import distance as dist
from imutils import face_utils
import time
import dlib
import cv2
import os
import sys
import collections
import random
import face_recognition
import pickle
import math
import threading
import tensorflow as tf

num_cores = 4
num_CPU = 1
num_GPU = 1

config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                                  inter_op_parallelism_threads=num_cores,
                                  allow_soft_placement=True,
                                  device_count={'CPU': num_CPU,
                                                'GPU': num_GPU}
                                  )

session = tf.compat.v1.Session(config=config)
K.set_session(session)


class FacialLandMarksPosition:
    """
    The indices points to the various facial features like left ear, right ear, nose, etc.,
    that are mapped from the Facial Landmarks used by dlib's FacialLandmarks predictor.
    """
    left_eye_start_index, left_eye_end_index = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    right_eye_start_index, right_eye_end_index = face_utils.FACIAL_LANDMARKS_IDXS[
        "right_eye"]


facial_landmarks_predictor = 'models/68_face_landmarks_predictor.dat'
predictor = dlib.shape_predictor(facial_landmarks_predictor)

modelForEye = load_model('models/weights.149-0.01.hdf5')


def predict_eye_state(model, image):
    image = cv2.resize(image, (20, 10))
    image = image.astype(dtype=np.float32)

    image_batch = np.reshape(image, (1, 10, 20, 1))
    image_batch = tf.keras.applications.mobilenet.preprocess_input(image_batch)

    return np.argmax(modelForEye.predict(image_batch)[0])

#cap = cv2.VideoCapture(0)


def eyeMain():
    try:
      left = None
      right = None
      scale = 0.5
      while(True):
          #c = time.time()

          # Capture frame-by-frame
          #ret, frame = cap.read()
          frame = cv2.imread('your_file_name.jpg')
          image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

          original_height, original_width = image.shape[:2]

          resized_image = cv2.resize(image,  (0, 0), fx=scale, fy=scale)
          lab = cv2.cvtColor(resized_image, cv2.COLOR_BGR2LAB)

          l, _, _ = cv2.split(lab)

          resized_height, resized_width = l.shape[:2]
          height_ratio, width_ratio = original_height / \
              resized_height, original_width / resized_width

          face_locations = face_recognition.face_locations(l, model='hog')

          if len(face_locations):
              top, right, bottom, left = face_locations[0]
              x1, y1, x2, y2 = left, top, right, bottom

              x1 = int(x1 * width_ratio)
              y1 = int(y1 * height_ratio)
              x2 = int(x2 * width_ratio)
              y2 = int(y2 * height_ratio)

          # draw face rectangle

              gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              shape = predictor(gray, dlib.rectangle(x1, y1, x2, y2))

              face_landmarks = face_utils.shape_to_np(shape)

              left_eye_indices = face_landmarks[FacialLandMarksPosition.left_eye_start_index:
                                                FacialLandMarksPosition.left_eye_end_index]

              (x, y, w, h) = cv2.boundingRect(np.array([left_eye_indices]))
              left_eye = gray[y:y + h, x:x + w]

              right_eye_indices = face_landmarks[FacialLandMarksPosition.right_eye_start_index:
                                                FacialLandMarksPosition.right_eye_end_index]

              (x, y, w, h) = cv2.boundingRect(np.array([right_eye_indices]))
              right_eye = gray[y:y + h, x:x + w]

              left_eye_open = 'yes' if predict_eye_state(
                  model=modelForEye, image=left_eye) else 'no'
              right_eye_open = 'yes' if predict_eye_state(
                  model=modelForEye, image=right_eye) else 'no'

              left = '{}'.format(left_eye_open)
              right = '{}'.format(right_eye_open)
          # print(right)
          #print('left eye open: {0}    right eye open: {1}'.format(left_eye_open, right_eye_open))
              break
              if left_eye_open == 'yes' and right_eye_open == 'yes':
                  cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
              else:
                  cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

          #cv2.imshow('right_eye', right_eye)
        # cv2.imshow('left_eye', left_eye)

          break

  # When everything done, release the capture
  # cap.release()
      cv2.destroyAllWindows()
      # print(left)
      # print(right)
      if(left == None or right == None):
          return None
      return{'left': left, 'right': right}
    except Exception:
      raise Exception("eyes not detected")


In [ ]:
# image_file = cv2.imread(r'your_file_name.jpg')
# result = FaceMain(image_file)
# print(result)

In [ ]:
# eyeMain()

In [ ]:
from datetime import datetime

def get_current_date():  
    """
    Get the current date in the format: DayName, DayWithOrdinalSuffix Month Year
    Example: 20th of May 2023
    """
    current_date = datetime.now()
    day = current_date.day
    ordinal_suffix = "th" if 11 <= day <= 13 else {1: "st", 2: "nd", 3: "rd"}.get(day % 10, "th")
    formatted_date = current_date.strftime(f"%d{ordinal_suffix} of %B %Y")
    return formatted_date


In [ ]:
import pytz

def get_current_time():
    """
    Get the current time in Lahore with AM/PM.
    Example: "ten thirty AM" or "ten thirty PM"
    """
    timezone = pytz.timezone('Asia/Karachi')
    current_time = datetime.now(timezone).time()
    hour = current_time.hour
    minute = current_time.minute
    
    # Convert hour to words
    if hour == 0:
        hour_word = "twelve"
    elif hour <= 12:
        hour_word = num_to_word(hour)
    else:
        hour_word = num_to_word(hour - 12)
    
    # Convert minute to words
    if minute == 0:
        minute_word = "o'clock"
    else:
        minute_word = num_to_word(minute)
    
    # Determine AM/PM
    am_pm = "AM" if hour < 12 else "PM"
    
    # Construct the time description
    time_description = f"{hour_word} {minute_word} {am_pm}"
    
    return time_description

def num_to_word(num):
    """
    Convert a number to its word representation.
    """
    units = [
        "zero", "one", "two", "three", "four", "five",
        "six", "seven", "eight", "nine", "ten", "eleven",
        "twelve", "thirteen", "fourteen", "fifteen", "sixteen",
        "seventeen", "eighteen", "nineteen"
    ]
    tens = [
        "", "", "twenty", "thirty", "forty", "fifty",
        "sixty", "seventy", "eighty", "ninety"
    ]
    
    if num < 20:
        return units[num]
    else:
        tens_digit = num // 10
        ones_digit = num % 10
        return f"{tens[tens_digit]} {units[ones_digit]}"

# Get the current time in Lahore

In [ ]:
import requests
import math

def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

def get_weather(city_name, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + city_name
    response = requests.get(complete_url)
    data = response.json()
    
    if data["cod"] != "404":
        main_info = data["main"]
        current_temperature = main_info["temp"]
        feels_like = main_info["feels_like"]
        current_pressure = main_info["pressure"]
        current_humidity = main_info["humidity"]
        weather_description = data["weather"][0]["description"]
        
        result = f"{weather_description} is currently in {city_name}. The current temperature in {city_name} is {math.floor(kelvin_to_celsius(current_temperature))} degrees Celsius, feels like {math.floor(kelvin_to_celsius(feels_like))} degrees Celsius. The atmospheric pressure is {current_pressure} hPa. The humidity level is {current_humidity}%."
        return result
    else:
        return None


In [ ]:
from deepface import DeepFace
obj = DeepFace.analyze(img_path="your_file_name.jpg",
                       actions=[ 'race', 'emotion'])
print(obj[0]['dominant_emotion'])
print(obj[0]['dominant_race'])

Directory  /root /.deepface created
Directory  /root /.deepface/weights created
facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 30.4MB/s]


race_model_single_batch.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/race_model_single_batch.h5
To: /root/.deepface/weights/race_model_single_batch.h5
100%|██████████| 537M/537M [00:08<00:00, 61.5MB/s]
Action: race:   0%|          | 0/2 [00:00<?, ?it/s]   

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Optional dependency 'langdetect' was used but it isn't installed.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Action: emotion:  50%|█████     | 1/2 [00:05<00:05,  6.00s/it]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Optional dependency 'langdetect' was used but it isn't installed.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Action: emotion: 100%|██████████| 2/2 [00:06<00:00,  3.17s/it]

angry
white


In [ ]:
import cv2
import numpy as np


def putlabels(x):

    if str(x) == '0':
        return 'neutral'
    elif str(x) == '1':
        return 'happy'
    elif str(x) == '2':
        return 'sad'
    elif str(x) == '3':
        return 'surprise'
    else:
        return 'anger'


def net_face(frame, shape):

    net = cv2.dnn.readNet(face_model + '.xml', face_model + '.bin')
    blob = cv2.dnn.blobFromImage(frame, size=shape, ddepth=cv2.CV_8U)
    net.setInput(blob)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    out = net.forward()
    return out


def net_emote(frame, shape):

    net = cv2.dnn.readNet(emote_model + '.xml', emote_model + '.bin')
    blob = cv2.dnn.blobFromImage(frame, size=shape, ddepth=cv2.CV_8U)
    net.setInput(blob)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    out = net.forward()
    return out


face_model = 'face-detection-retail-0005'
emote_model = 'emotions-recognition-retail-0003'
path = 'in.mp4'

#cap = cv2.VideoCapture(0)


def mainForEmotion():
    frame = cv2.imread("your_file_name.jpg")
#frame_width = int(cap.get(3))
#frame_height = int(cap.get(4))


#writer = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame_width,frame_height))
    returnvalue = None
    while(True):
        #ret, frame = cap.read()

        # if ret == False:
        #print('End of video')
        # break

        out = net_face(frame, (300, 300))

        for detection in out.reshape(-1, 7):

            conf = float(detection[2])
            xmin = int(detection[3] * frame.shape[1])
            ymin = int(detection[4] * frame.shape[0])
            xmax = int(detection[5] * frame.shape[1])
            ymax = int(detection[6] * frame.shape[0])

            if conf > 0.65:

                soft = net_emote(frame, (64, 64))
                emo_pred = np.reshape(soft, (5,))
                emo_pred = list(emo_pred)
                returnvalue = putlabels(emo_pred.index(max(emo_pred)))
            #cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,128,255), 2)
            #cv2.putText(frame, putlabels(emo_pred.index(max(emo_pred))) , (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # cv2.imshow('Emotion-Recognition',frame)
        break
    # writer.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    return returnvalue
    # print(returnvalue)

# cap.release()
    cv2.destroyAllWindows()

# frame=cv2.imread("your_file_name.jpg")
# #frame_width = int(cap.get(3))
# #frame_height = int(cap.get(4))

# #writer = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame_width,frame_height))
# returnvalue=None
# while(True):
#     #ret, frame = cap.read()

#     #if ret == False:
#         #print('End of video')
#        # break

#     out = net_face(frame, (300,300))

#     for detection in out.reshape(-1,7):

#         conf = float(detection[2])
#         xmin = int(detection[3] * frame.shape[1])
#         ymin = int(detection[4] * frame.shape[0])
#         xmax = int(detection[5] * frame.shape[1])
#         ymax = int(detection[6] * frame.shape[0])

#         if conf > 0.65:

#             soft = net_emote(frame,(64,64))
#             emo_pred = np.reshape(soft, (5,))
#             emo_pred = list(emo_pred)
#             returnvalue=putlabels(emo_pred.index(max(emo_pred)))
#             #cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,128,255), 2)
#             #cv2.putText(frame, putlabels(emo_pred.index(max(emo_pred))) , (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

#         #cv2.imshow('Emotion-Recognition',frame)
#     break
#     #writer.write(frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# print(returnvalue)
# #cap.release()
# cv2.destroyAllWindows()
# #writer.release()


In [ ]:
!pip3 install opencv-python-inference-engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 5.7 MB/s eta 0:00:00


In [ ]:
import string
import random
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import load_model

nltk.download("punkt")
nltk.download("wordnet")



In [ ]:
# Load the saved model
ourNewModel = load_model('/content/drive/MyDrive/gfgModelFinal.h5')

In [ ]:
import json
# File path of the JSON file
file_path = '/content/drive/MyDrive/intents.json'
with open(file_path) as file:
    ourData = json.load(file)

In [ ]:
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)# tokenize the patterns
        newWords.extend(ournewTkns)# extends the tokens
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [ ]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  # print(num.array([bagOwords]))
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  print(ourResult)
  newThresh = 0.9
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [ ]:
def convert_to_lowercase(string):
    lowercase_string = string.lower()
    return lowercase_string


In [ ]:
def askBot(newMessage):
  # Example usage
    newMessage = convert_to_lowercase(newMessage)
    intents = Pclass(newMessage, newWords, ourClasses)
    if intents:
      ourResult = getRes(intents, ourData)
      if ourResult=="":
        return False
      else:
        return ourResult
    else:
        return False

In [ ]:
from deepface import DeepFace


def getRace():
    try:
        obj = DeepFace.analyze(img_path="your_file_name.jpg", actions=['race'])
        return obj[0]["dominant_race"]

    except Exception:
        raise Exception("face not detected")
def getEmotion():
    try:
      obj = DeepFace.analyze(img_path="your_file_name.jpg",
                              actions=['emotion'])
      return obj[0]["dominant_emotion"]
    except Exception:
        raise Exception("face not detected")

In [ ]:
!pip install -U flask-cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import io
import json
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
from threading import Thread
from flask import Flask, jsonify, request,make_response
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
app = Flask(__name__)
CORS(app)
# run_with_ngrok(app)
@app.route('/race', methods=['POST'])
def predict():
    if request.method == 'POST':
      try:
        file = request.files['file']
        file.save("your_file_name.jpg")
        result=getRace()
        return jsonify({'Race': result})      
      except Exception as e:
        response = make_response(jsonify({'message': str(e)}), 400)
        return response

@app.route('/emotion', methods=['POST'])
def predictEmotion():
    if request.method == 'POST':
      try:
        file = request.files['file']
        file.save("your_file_name.jpg")
        result=getEmotion()
        return jsonify({'emotion': result})
      except Exception as e:
        response = make_response(jsonify({'message': str(e)}), 400)
        return response

@app.route('/faceGenderAge', methods=['POST'])
def predictFaceAgeGender():
    if request.method == 'POST':
      try:
        file = request.files['file']
        file.save("your_file_name.jpg")
        image_file = cv2.imread(r'your_file_name.jpg')
        download_and_append_encodings()
        result=FaceMain(image_file)
        if(not result):
            response = make_response(jsonify({'message': "Could not detect face"}), 400)
            return response
        object_data = json.loads(str(result['name']))
        # Access the values in the dictionary
        name = object_data['name']

        return jsonify({'result':{'name': name,
                        'age': str(result['age']),
                        'gender': str(result['gender'])}})
      except Exception as e:
        response = make_response(jsonify({'message': str(e)}), 400)
        return response

@app.route('/eye', methods=['POST'])
def predictEye():
    if request.method == 'POST':
      try:
        file = request.files['file']
        result=eyeMain()
        if(not result):
            response = make_response(jsonify({'message': "Could not detect eyes"}), 400)
            return response
        return {'result':result}
      except Exception as e:
        response = make_response(jsonify({'message': str(e)}), 400)
        return response
          
@app.route('/date', methods=['POST'])
def getDate():
  if request.method == 'POST':
    try:
      result=get_current_date()
      return jsonify({'date': result})
    except:
      response = make_response(jsonify({'message': "could not get date something bad happend"}), 400)
      return response

@app.route('/time', methods=['POST'])
def getTime():
  if request.method == 'POST':
    try:
      result=  get_current_time()
      return jsonify({'time': result})
    except:
      response = make_response(jsonify({'message': "could not get date something bad happend"}), 400)
      return response

@app.route('/weather', methods=['POST'])
def getWeather():
  if request.method == 'POST':
    try:
      api_key = "put-your-own-open-weather-api-key"
      city_name = "Lahore"
      weather_info = get_weather(city_name, api_key)
      return jsonify({"weather": weather_info})
    except:
      response = make_response(jsonify({'message': "could not get date something bad happend"}), 400)
      return response

@app.route('/qa', methods=['POST'])
def getQA():
  if request.method == 'POST':
    try:
      question = request.form['name']
      result=askQuestion(question)
      # Destructure the dictionary
      question, answer, score = result['question'], result['answer'], result['score']
      return jsonify({'answer': answer,'score':score})
    except Exception as e:
      response = make_response(jsonify({'message': str(e)}), 400)
      return response

@app.route('/studentAttendance', methods=['POST'])
def storeAttendance():
    if request.method == 'POST':
      try:
        file = request.files['file']
        file.save("your_file_name.jpg")
        image_file = cv2.imread(r'your_file_name.jpg')
        # img_bytes = file.read()
        download_and_append_encodings()
        result=FaceMain(image_file)

        if(not result):
            response = make_response(jsonify({'message': "Could not detect your face."}), 400)
            return response

        object_data = json.loads(str(result['name']))
        # Access the values in the dictionary
        name = object_data['name']
        student_id = object_data['student_id']
        Attendance.create_attendance(student_id)
        return jsonify({'message': f'{name} your Attendance has been marked'})
      except Exception as e:
        response = make_response(jsonify({'message': str(e)}), 400)
        return response


@app.route('/webGpt', methods=['POST'])
def qaFromWeb():
  try:
      question = request.form['question']
      answer = askBot(question)
      if answer=='apply date model':
        try:
          result=get_current_date()
          return jsonify({'answer': result})
        except:
          response = make_response(jsonify({'message': "could not get date something bad happend"}))
          return response      
      elif answer=="apply weather model":
        try:
          api_key = "put-your-own-open-weather-api-key"
          city_name = "Lahore"
          weather_info = get_weather(city_name, api_key)
          return jsonify({"weather": weather_info})
        except:
          response = make_response(jsonify({'message': "could not get date something bad happend"}), 400)
          return response
  
      elif answer=="apply time model":
        try:
          result=  get_current_time()
          return jsonify({'answer': result})
        except:
          response = make_response(jsonify({'message': "could not get date something bad happend"}))
          return response
      elif answer == "apply name model" :
        response = make_response(jsonify({'answer': "I'm sorry, but as an AI language model, I don't have access to personal information like name"}))
        return response
      elif answer=="apply age model":
        response = make_response(jsonify({'answer': "I'm sorry, but as an AI language model, I don't have access to personal information like age"}))
        return response
      elif answer=="apply gender model":
        response = make_response(jsonify({'answer': "I'm sorry, but as an AI language model, I don't have access to personal information like gender"}))
        return response
      
      elif answer=="apply emotion model":
        response = make_response(jsonify({'answer': "I'm sorry, but as an AI language model, I don't have access to personal information like emotion"}))
        return response

      elif answer=="apply race model":
        response = make_response(jsonify({'answer': "I'm sorry, but as an AI language model, I don't have access to personal information like race"}))
        return response

      elif answer=="apply attendance model":
        response = make_response(jsonify({'answer': "I'm sorry, but as an AI language model, I don't have access to personal information like face"}))
        return response

      elif answer=="apply eye model":
        response = make_response(jsonify({'answer': "I'm sorry, but as an AI language model, I don't have access to personal information like eye"}))
        return response
      elif not answer:
        try:
          result=askQuestion(question)
          # Destructure the dictionary
          question, answer, score = result['question'], result['answer'], result['score']
          return jsonify({'answer': answer,'score':score})
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}))
          return response
      else:
        return jsonify({'answer': answer})
  except:
    response = make_response(jsonify({'message': "could not get answer something bad happend"}), 400)
    return response

@app.route('/suhaBot', methods=['POST'])
def qaFromSuha():
  try:
      question = request.form['question']
      answer = askBot(question)
      if answer=='apply date model':
        try:
          result=get_current_date()
          return jsonify({'answer': result})
        except:
          response = make_response(jsonify({'message': "could not get date something bad happend"}))
          return response      
      elif answer=="apply weather model":
        try:
          api_key = "put-your-own-open-weather-api-key"
          city_name = "Lahore"
          weather_info = get_weather(city_name, api_key)
          return jsonify({"answer": weather_info})
        except:
          response = make_response(jsonify({'message': "could not get date something bad happend"}), 400)
          return response
  
      elif answer=="apply time model":
        try:
          result=  get_current_time()
          return jsonify({'answer': result})
        except:
          response = make_response(jsonify({'message': "could not get date something bad happend"}))
          return response
          
      elif answer == "apply name model" :
        try:
          file = request.files['file']
          file.save("your_file_name.jpg")
          image_file = cv2.imread(r'your_file_name.jpg')
          download_and_append_encodings()
          result=FaceMain(image_file)
          if(not result):
              response = make_response(jsonify({'message': "We do not know each other"}), 400)
              return response
          object_data = json.loads(str(result['name']))
          # Access the values in the dictionary
          name = object_data['name']
          response = make_response(jsonify({'answer': f"We know each other your name is {name}."}))
          return response
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}), 400)
          return response        

      elif answer=="apply age model":
        try:
          file = request.files['file']
          file.save("your_file_name.jpg")
          image_file = cv2.imread(r'your_file_name.jpg')
          download_and_append_encodings()
          result=FaceMain(image_file)
          if(not result):
              response = make_response(jsonify({'message': "Could not detect your face"}), 400)
              return response
          object_data = json.loads(str(result['name']))
          # Access the values in the dictionary
          name = object_data['name']
          age=result['age']
          response =make_response(jsonify({'answer': f'Looks like you are {age} Years old'}))
          return response
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}), 400)
          return response        

      elif answer=="apply gender model":
        try:
          file = request.files['file']
          file.save("your_file_name.jpg")
          image_file = cv2.imread(r'your_file_name.jpg')
          download_and_append_encodings()
          result=FaceMain(image_file)
          if(not result):
              response = make_response(jsonify({'message': "Could not detect your face"}), 400)
              return response
          object_data = json.loads(str(result['name']))
          # Access the values in the dictionary
          name = object_data['name']
          gender=result['gender']
          genderToReturn="Female"
          if gender== 1:
            genderToReturn="Male"
          response =make_response(jsonify({'answer': f'You are {genderToReturn}'}))
          return response
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}), 400)
          return response        
      
      elif answer=="apply emotion model":
        try:
          file = request.files['file']
          file.save("your_file_name.jpg")
          result=getEmotion()
          response =make_response(jsonify({'answer': f'your gaze looks like you are {result}'}))
          return response
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}), 400)
          return response

      elif answer=="apply race model":
        try:
          file = request.files['file']
          file.save("your_file_name.jpg")
          result=getRace()
          response =make_response(jsonify({'answer': f'Looks like you are {result}'}))
          return response
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}), 400)
          return response

      elif answer=="apply attendance model":
        try:
          file = request.files['file']
          file.save("your_file_name.jpg")
          image_file = cv2.imread(r'your_file_name.jpg')
          download_and_append_encodings()
          result=FaceMain(image_file)

          if(not result):
              response = make_response(jsonify({'message': "Could not detect your face."}), 400)
              return response

          object_data = json.loads(str(result['name']))
          # Access the values in the dictionary
          name = object_data['name']
          student_id = object_data['student_id']
          Attendance.create_attendance(student_id)
          return jsonify({'answer': f'{name} your Attendance has been marked'})
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}), 400)
          return response


      elif answer=="apply eye model":
        try:
          file = request.files['file']
          result=eyeMain()
          if(not result):
              response = make_response(jsonify({'message': "Could not detect eyes"}), 400)
              return response
          left_eye = result["left"]
          right_eye = result["right"]
          left_eye_speak = "opened" if left_eye == "yes" else "closed"
          right_eye_speak = "opened" if right_eye == "yes" else "closed"          
          response =make_response(jsonify({'answer': f'"Your left eye is {left_eye_speak} and Your Right eye is {right_eye_speak} ."'}))
          return response

        except Exception as e:
          response = make_response(jsonify({'message': str(e)}), 400)
          return response
      elif not answer:
        try:
          result=askQuestion(question)
          # Destructure the dictionary
          question, answer, score = result['question'], result['answer'], result['score']
          return jsonify({'answer': answer,'score':score})
        except Exception as e:
          response = make_response(jsonify({'message': str(e)}))
          return response
      else:
        return jsonify({'answer': answer})
  except:
    response = make_response(jsonify({'message': "could not get answer something bad happend"}), 400)
    return response


def run_flask_app():
    app.run(port=5002)  # Replace with your desired port number

# Start the Flask application in a separate thread
flask_thread = Thread(target=run_flask_app)
flask_thread.start()



In [ ]:
import requests

url = 'http://localhost:5002/studentAttendance'  # Replace with your Flask app URL
payload = {'question': "what is ?"}
# Open the image file
with open('your_file_name.jpg', 'rb') as file:
    # Send a POST request with the file as multipart form data
    response = requests.post(url,data=payload, files={'file': file})
# response = requests.post(url, data=payload)
# Check the response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.json()}")


INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:12:49] "POST /studentAttendance HTTP/1.1" 200 -


{'message': 'Ahmed Hassan your Attendance has been marked'}


In [ ]:
!ngrok http 5002 --log=stdout

INFO[05-23|05:12:49] no configuration paths supplied 
INFO[05-23|05:12:49] using configuration at default config path path=/root/.config/ngrok/ngrok.yml
INFO[05-23|05:12:49] open config file                         path=/root/.config/ngrok/ngrok.yml err=nil
t=2023-05-23T05:12:49+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2023-05-23T05:12:49+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=8a42acc4519b
t=2023-05-23T05:12:49+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2023-05-23T05:12:49+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:5002 url=https://c482-34-32-211-192.ngrok-free.app


INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:25:17] "OPTIONS /webGpt HTTP/1.1" 200 -


t=2023-05-23T05:25:17+0000 lvl=info msg="join connections" obj=join id=18cbb6fb39fd l=127.0.0.1:5002 r=223.123.46.164:5732
1/1 [==============================] - 0s 21ms/step
[4.4392175e-03 1.3617949e-03 1.5415341e-02 1.0139704e-02 4.5269150e-02
 1.7727184e-03 3.8575295e-02 3.7303595e-03 3.1855813e-01 2.9444168e-04
 1.7410979e-03 1.0623328e-03 3.8908686e-02 4.2379410e-03 4.5149531e-03
 1.6621668e-03 6.5770693e-02 2.6355882e-03 7.0929909e-03 2.2658186e-02
 1.3936204e-03 5.3968523e-03 3.6525518e-02 1.4794647e-05 4.9261484e-02
 3.2373238e-03 2.3671277e-03 8.1434650e-03 2.6128518e-03 1.1795074e-03
 2.4922567e-03 2.2673702e-03 2.5812178e-03 1.1032979e-03 6.8031080e-02
 2.7706096e-04 1.6035747e-03 1.1106090e-03 3.0795410e-02 3.0801653e-05
 2.8945706e-03 1.1647121e-03 4.7898509e-02 8.2053123e-03 1.5746489e-02
 1.0859592e-01 5.2284594e-03]
t=2023-05-23T05:25:17+0000 lvl=info msg="join connections" obj=join id=0a79e877894d l=127.0.0.1:5002 r=223.123.46.164:5732


INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:25:19] "POST /webGpt HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:27:35] "OPTIONS /webGpt HTTP/1.1" 200 -


t=2023-05-23T05:27:35+0000 lvl=info msg="join connections" obj=join id=6fc16338e0b9 l=127.0.0.1:5002 r=223.123.46.164:5732
t=2023-05-23T05:27:36+0000 lvl=info msg="join connections" obj=join id=5138a27f269b l=127.0.0.1:5002 r=223.123.46.164:5732
1/1 [==============================] - 0s 24ms/step
[4.4392175e-03 1.3617949e-03 1.5415341e-02 1.0139704e-02 4.5269150e-02
 1.7727184e-03 3.8575295e-02 3.7303595e-03 3.1855813e-01 2.9444168e-04
 1.7410979e-03 1.0623328e-03 3.8908686e-02 4.2379410e-03 4.5149531e-03
 1.6621668e-03 6.5770693e-02 2.6355882e-03 7.0929909e-03 2.2658186e-02
 1.3936204e-03 5.3968523e-03 3.6525518e-02 1.4794647e-05 4.9261484e-02
 3.2373238e-03 2.3671277e-03 8.1434650e-03 2.6128518e-03 1.1795074e-03
 2.4922567e-03 2.2673702e-03 2.5812178e-03 1.1032979e-03 6.8031080e-02
 2.7706096e-04 1.6035747e-03 1.1106090e-03 3.0795410e-02 3.0801653e-05
 2.8945706e-03 1.1647121e-03 4.7898509e-02 8.2053123e-03 1.5746489e-02
 1.0859592e-01 5.2284594e-03]


...

INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:27:52] "POST /webGpt HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:28:09] "OPTIONS /webGpt HTTP/1.1" 200 -


t=2023-05-23T05:28:09+0000 lvl=info msg="join connections" obj=join id=08335e0e7f86 l=127.0.0.1:5002 r=223.123.46.164:5732
t=2023-05-23T05:28:10+0000 lvl=info msg="join connections" obj=join id=5a4148d5e89d l=127.0.0.1:5002 r=223.123.46.164:5732
1/1 [==============================] - 0s 20ms/step
[4.4392175e-03 1.3617949e-03 1.5415341e-02 1.0139704e-02 4.5269150e-02
 1.7727184e-03 3.8575295e-02 3.7303595e-03 3.1855813e-01 2.9444168e-04
 1.7410979e-03 1.0623328e-03 3.8908686e-02 4.2379410e-03 4.5149531e-03
 1.6621668e-03 6.5770693e-02 2.6355882e-03 7.0929909e-03 2.2658186e-02
 1.3936204e-03 5.3968523e-03 3.6525518e-02 1.4794647e-05 4.9261484e-02
 3.2373238e-03 2.3671277e-03 8.1434650e-03 2.6128518e-03 1.1795074e-03
 2.4922567e-03 2.2673702e-03 2.5812178e-03 1.1032979e-03 6.8031080e-02
 2.7706096e-04 1.6035747e-03 1.1106090e-03 3.0795410e-02 3.0801653e-05
 2.8945706e-03 1.1647121e-03 4.7898509e-02 8.2053123e-03 1.5746489e-02
 1.0859592e-01 5.2284594e-03]
t=2023-05-23T05:28:12+0000 lvl=in

INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:28:17] "POST /webGpt HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:28:22] "POST /webGpt HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 05:28:23] "POST /webGpt HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/May/2023 06:03:02] "OPTIONS /webGpt HTTP/1.1" 200 -


t=2023-05-23T06:03:02+0000 lvl=info msg="join connections" obj=join id=c5db55199892 l=127.0.0.1:5002 r=223.123.46.164:5644
1/1 [==============================] - 0s 20ms/step
[4.4392175e-03 1.3617949e-03 1.5415341e-02 1.0139704e-02 4.5269150e-02
 1.7727184e-03 3.8575295e-02 3.7303595e-03 3.1855813e-01 2.9444168e-04
 1.7410979e-03 1.0623328e-03 3.8908686e-02 4.2379410e-03 4.5149531e-03
 1.6621668e-03 6.5770693e-02 2.6355882e-03 7.0929909e-03 2.2658186e-02
 1.3936204e-03 5.3968523e-03 3.6525518e-02 1.4794647e-05 4.9261484e-02
 3.2373238e-03 2.3671277e-03 8.1434650e-03 2.6128518e-03 1.1795074e-03
 2.4922567e-03 2.2673702e-03 2.5812178e-03 1.1032979e-03 6.8031080e-02
 2.7706096e-04 1.6035747e-03 1.1106090e-03 3.0795410e-02 3.0801653e-05
 2.8945706e-03 1.1647121e-03 4.7898509e-02 8.2053123e-03 1.5746489e-02
 1.0859592e-01 5.2284594e-03]
t=2023-05-23T06:03:03+0000 lvl=info msg="join connections" obj=join id=625927872b6d l=127.0.0.1:5002 r=223.123.46.164:5644


INFO:werkzeug:127.0.0.1 - - [23/May/2023 06:03:05] "POST /webGpt HTTP/1.1" 200 -


## **Open new Colab Window and run the following blocks**

1. Get our test image.
```
!wget https://raw.githubusercontent.com/rajeevratan84/ModernComputerVision/main/bird.jpg
```

2. Send our API Post Requests. Replace the URL below with the NGROK URL above.
```
import requests
resp = requests.post("http://xxxxxxxxxxxx.ngrok.io/predict",
                     files={"file": open('bird.jpg','rb')})
print(resp.json())
```


